# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window

spark = SparkSession.builder.appName("Quiz Spark SQL").getOrCreate()



In [2]:
path = "data/sparkify_log_small.json"
sdf_user = spark.read.json(path)

In [3]:
sdf_user.createOrReplaceTempView("sdf_user_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [10]:
# TODO: write your code to answer question 1
spark.sql('''
    SELECT DISTINCT page
    FROM sdf_user_table
    WHERE page NOT IN (
        SELECT DISTINCT page
        FROM sdf_user_table
        WHERE userId = ""
    )
''').show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [6]:
# TODO: write your code to answer question 3
spark.sql('''
        SELECT COUNT (DISTINCT userId)
        FROM sdf_user_table
        WHERE gender = 'F'
''').show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [13]:
# TODO: write your code to answer question 4
spark.sql('''
        SELECT artist, COUNT(*) as played_artist
        FROM sdf_user_table
        WHERE page = 'NextSong'
        GROUP BY artist
        ORDER BY played_artist DESC
        LIMIT 1
''').show()

+--------+-------------+
|  artist|played_artist|
+--------+-------------+
|Coldplay|           83|
+--------+-------------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [61]:
# TODO: write your code to answer question 5

visit_home = spark.sql('''
    SELECT userId, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS visit_home
    FROM sdf_user_table
    WHERE page in ("Home", "NextSong") --AND userID = '1138' 
''')

visit_home.createOrReplaceTempView("visit_home_table")

rank_visit_home = spark.sql('''
    SELECT userId, page, SUM(visit_home)
    OVER(PARTITION BY userId ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rank_visit_home
    FROM visit_home_table
''')

rank_visit_home.createOrReplaceTempView("rank_visit_home_table")

spark.sql('''
    SELECT AVG(count_num_songs) avg_num_songs
    FROM(
        SELECT COUNT(rank_visit_home) count_num_songs
        FROM rank_visit_home_table
        WHERE page = 'NextSong'
        GROUP BY userId, rank_visit_home
    )
''').collect()

[Row(avg_num_songs=6.898347107438017)]